In [1]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
import io
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import warnings
warnings.filterwarnings("ignore")

In [2]:
import lightgbm as lgb
import xgboost as xgb
import catboost as catb
from catboost import CatBoostClassifier

#from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

In [3]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

data = pd.concat([train_data, test_data]).reset_index().drop(['index'], axis=1)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
torch = pd.read_csv('torch.csv')
data['torch'] = np.nan
data.loc[891:1308, 'torch']= torch['torch'].values

In [5]:
data['Surname'] = data['Name'].apply(lambda x: x.split(',')[0])

In [6]:
# New Ticket_id column
data['Ticket_id'] = 'new_col'
# Initialize Ticket_id = Pclass + Ticket + Fare + Embarked
def ticket_id(row):
    row['Ticket_id'] = str(row.Pclass) + '-' + str(row.Ticket)[:-1] + '-' + str(row.Fare) + '-' + str(row.Embarked)
    return row
data = data.apply(ticket_id, axis='columns')

In [7]:
# New Group_id column
data['Group_id'] = 'new_col2'
# Initialize Group_id = Surname + Ticket_id
def group_id(row):
    row['Group_id'] = str(row.Surname) + '-' + str(row.Ticket_id)
    return row

data = data.apply(group_id, axis='columns')

In [8]:
# creation of the Title feature
data['Title'] = 'man'
data.loc[data.Sex == 'female', 'Title'] = 'woman'
data.loc[data['Name'].str.contains('Master'), 'Title'] = 'boy'

In [9]:
data.loc[data.Title == 'man', 'Group_id'] = 'noGroup'
# New column with WC frequency
data['WC_count'] = data.loc[data.Title != 'man'].groupby('Group_id')['Group_id'].transform('count')
# assign noGroup to every unique value
data.loc[data.WC_count <=1, 'Group_id'] = 'noGroup'

In [10]:
indices = []
count = 0
for i in range(0,1309):
    if (data.loc[i,'Title'] != 'man') & (data.loc[i,'Group_id'] == 'noGroup'):
        data.loc[i,'Group_id'] = data.loc[(data['Ticket_id'] == data.loc[i, 'Ticket_id']) & (data.Title != 'man'), 'Group_id'].iloc[0]
        if (data.loc[i, 'Group_id'] != 'noGroup'):
            indices.append(i)
            count += 1
print('{:d} passengers were added to an existing group'.format(count))

11 passengers were added to an existing group


In [11]:
data['WCSurvived'] = data.loc[(data.Title != 'man') & (data.Group_id != 'noGroup')].groupby('Group_id').Perished.transform('mean')

In [12]:
# Get the family names using set difference
test_groups = set(data[891:1309].Group_id.unique()) - set(data[0:891].Group_id.unique())

In [13]:
# Assign WCSurvived = 0 to 3rd class test families, else 1
data.loc[data.Group_id.isin(test_groups), 'WCSurvived'] = 0
data.loc[(data.Group_id.isin(test_groups)) & (data.Pclass != 3), 'WCSurvived'] = 1

In [15]:
# 欠損値の補完
x = data.loc[(data['Pclass'] == 3) & 
       (data['Sex'] == 'male' ) &
       (data['Embarked'] == 'S') & 
       (data['Title'] == 'Mr' ) &
       (data['Fare'] < 10 ) &
        (data['Fare'] > 5 ) ,
        'Fare']
data.Fare.fillna(data.mean(), inplace=True)

In [16]:
# Assign np.NaN to zero-fares
def fix_fare(row):
    if row.Fare == 0:
        row.Fare = np.NaN
    return row
# Apply the fix_fare function 
data = data.apply(fix_fare, axis='columns')

In [17]:
def Preprocessing(dataset):
    #SibSpとParchの処理
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    #FamilySizeの分類
    dataset['SmallFamily'] = dataset['FamilySize'].map(lambda s: 1if 2 <= s <= 4 else 0)
    dataset['LargeFamily'] = dataset['FamilySize'].map(lambda s: 1 if  5 <=s else 0)
    #Cabinの処理
    cabin_only = dataset[["Cabin"]].copy()
    cabin_only["Cabin_Data"] = cabin_only["Cabin"].isnull().apply(lambda x: not x)
    cabin_only["Deck"] = cabin_only["Cabin"].str.slice(0,1)
    cabin_only["Deck"] = cabin_only["Deck"].fillna("N")
    cabin_only["Deck"] = cabin_only['Deck'].replace('T','A')
    cabin_only["Deck"] = cabin_only['Deck'].replace('G','F')
    dataset = pd.concat([ dataset,cabin_only["Deck"]], axis=1)
    
    Ticket = []
    for i in list(dataset.Ticket):
        if not i.isdigit() :
            Ticket.append(i.replace(".","").replace("/","").strip().split(' ')[0]) #Take prefix
        else:
            Ticket.append("X")

    dataset["Ticket"] = Ticket
    
    return dataset
data = Preprocessing(data)
data['FirstName'] = data['Name'].str.extract('(Mr\. |Miss\. |Master. |Mrs\.[A-Za-z ]*\()([A-Za-z]*)')[1]
chang_list = ['SCA3', 'STONOQ', 'AQ4', 'A', 'LP', 'AQ3', 'SP', 'SOP', 'Fa', 'LINE',
              'SWPP', 'SCOW', 'LINE', 'LINE', 'PPP', 'AS', 'SWPP', 'LINE', 'PPP', 'CASOTON','SCA4','SCParis', 'WEP',
              'SCPARIS', 'A4', 'FC','STONO2', 'SOC', 'A5', 'STONO2', 'STONO2', 'C', 'SC', 'A5', 'STONO2', 'PP',
             ]
data['Ticket'] = data['Ticket'].replace(chang_list, 'X')

In [18]:
for j in data[data['Fare'].isnull()].index:
    for i in data[['Pclass', 'Sex', 'Embarked', 'Title', 'Deck']][data.index == j].values:
        data['Fare'].loc[j] = data['Fare'][
            (data['Pclass'] == i[0])&
            (data['Sex'] == i[1])&
            (data['Embarked'] == i[2])&
            (data['Title'] == i[3])&
            (data['Deck'] == i[4])].median()

In [19]:
# Calculate Ticket frequency and divide Fare by it
data['Ticket_freq'] = data.groupby('Ticket')['Ticket'].transform('count')
data['Pfare'] = data['Fare'] / data['Ticket_freq']

## Explore non-WCG females
Non-WCG females are a very wide group which contains solo females but also women traveling with their spouse.  
A way to isolate them from the dataframe is of course select null `WCSurvived` entries from the female subpopulation.

In [20]:
def Le_SS(dataset, category, numerical_cols):
    Le = LabelEncoder()
    for i in  category:
        dataset[i] = Le.fit_transform(dataset[i])
    SS = StandardScaler()
    for i in numerical_cols:
        dataset[i] = SS.fit_transform(dataset[i].values.reshape(-1, 1))
    return dataset

def dumm_SS(dataset, category, numerical_cols):
    dataset = pd.get_dummies(dataset, columns=category)
    SS = StandardScaler()
    for i in numerical_cols:
        dataset[i] = SS.fit_transform(dataset[i].values.reshape(-1, 1))
    return dataset

# データから予測

In [21]:
data['Predict'] = np.nan
data['Predict'][(data.Sex == 'female') & (data['WCSurvived'] == 0)] = 0
data['Predict'][(data.Sex == 'female') & (data['WCSurvived'] == 1)] = 1
data['Predict'][(data.Title == 'boy') & (data['WCSurvived'] == 1) & (data['WCSurvived'] == 0.25)] = 1
# これから先が難しい，よう変更
# データが少ない過学習に陥る、femaleの予測が難しい　予測しようとすると全て同じ値を返してしまう
# 残ったデータでは男は1が多くて→機会学習で予測はいい感じ
# 女は0が多い予想が難しい。データの少なさに起因。なのでなるべく手動で分類したい　80%以上は全て手動にする？？
# いい分類方法がわからない
#いいクラス,DeckがNではないものは0にする→これをしてしまうとデータが少なすぎて予測ができない
# data['Predict'][(data['Predict'].isnull()) & (data.Sex == 'female') & (data.Deck != 'N') ] = 0
# data['Predict'][(data['Predict'].isnull()) & (data.Sex == 'female') & (data.Pclass == 1) ] = 0

In [22]:
a = data.loc[891:1308, 'Perished'][(data['Predict'].isnull() == False)]
b = data.loc[891:1308, 'Predict'][(data['Predict'].isnull() == False)]
print( (a == b).sum() / len(a) )

0.8888888888888888


In [23]:
train = data[0:891][(data['Predict'].isnull()) & (data.Sex == 'male')]
test = data[891:1309][(data['Predict'].isnull()) & (data.Sex == 'male')]

numerical_cols = ['Pfare','Fare']
category = ['Pclass', 'Embarked','Ticket', 'Deck','Title']
features = ['Fare','Pfare', 'Pclass', 'Embarked', 'Deck','Ticket','Title']

# numerical_cols = ['Fare']#'Pfare
# category = ['Pclass', 'Embarked','Ticket', 'Deck','SmallFamily','LargeFamily','Ticket_freq']
# features = ['Fare', 'Pclass', 'Embarked', 'Deck','SmallFamily','LargeFamily','Ticket','Ticket_freq']#'Pfare

y_train = train['Perished']
X_train = train[features]
X_test = test[features]

X_train = dumm_SS(X_train, category, numerical_cols)
X_test = dumm_SS(X_test , category, numerical_cols)

X_train = np.array(X_train)
X_test = np.array(X_test)
k_male = pd.DataFrame()
for k in [9,11,7]:
    KC = KNeighborsClassifier(n_neighbors=k)
    KC.fit(X_train,y_train)
    predictions = KC.predict(X_test)
    k_male[k] = predictions

In [24]:
(k_male.mode(axis=1)[0].values.astype(int) == data.loc[891:1308, 'Perished'][(data['Predict'].isnull()) & (data.Sex == 'male')]).sum() / X_test.shape[0]

0.8345864661654135

In [25]:
data.loc[891:1309,'Predict'][(data['Predict'].isnull()) & (data.Sex == 'male')]=k_male.mode(axis=1)[0].values.astype(int)

# female

In [26]:
train = data[0:891]#[(data['Predict'].isnull())]# & (data.Sex == 'female')
test = data[891:1309][(data['Predict'].isnull()) & (data.Sex == 'female')]
numerical_cols = ['Pfare','Fare']
category = ['Pclass', 'Embarked','Ticket', 'Deck','Sex','Title']
features = ['Fare','Pfare', 'Pclass', 'Embarked', 'Deck','Ticket','Sex','Title']

y_train = train['Perished']
X_train = train[features]
X_test = test[features]

X_train = dumm_SS(X_train, category, numerical_cols)
X_test = dumm_SS(X_test , category, numerical_cols)
X_train = X_train.drop(columns=['Ticket_SOTONO2', 'Ticket_STONO', 'Sex_male', 'Title_boy', 'Title_man'])
X_train = np.array(X_train)
X_test = np.array(X_test)

In [27]:
k_male = pd.DataFrame()
for k in [2,3]:
    KC = KNeighborsClassifier(n_neighbors=k)
    KC.fit(X_train,y_train)
    predictions = KC.predict(X_test)
    k_male[k] = predictions
k_male['torch'] = data.loc[891:1308, 'torch'][(data['Predict'].isnull()) & (data.Sex == 'female')]

In [28]:
data.loc[891:1308, 'Predict'][(data['Predict'].isnull()) & (data.Sex == 'female')] = k_male.mode(axis=1)[0].values.astype(int)

In [29]:
#data.loc[891:1308, 'Predict'][(data['Predict'].isnull()) & (data.Sex == 'female')] = data.loc[891:1308, 'torch'][(data['Predict'].isnull()) & (data.Sex == 'female')]

In [30]:
# これから先が難しい，よう変更
# データが少ない過学習に陥る、femaleの予測が難しい　予測しようとすると全て同じ値を返してしまう
# 残ったデータでは男は1が多くて→機会学習で予測はいい感じ
# 女は0が多い予想が難しい。データの少なさに起因。なのでなるべく手動で分類したい　80%以上は全て手動にする？？
#女は１を予想するのが難しい
# いい分類方法がわからない
#いいクラス,DeckがNではないものは0にする→これをしてしまうとデータが少なすぎて予測ができない
#以下の処理をしてもあまり精度は向上しない
#以下のものは予測ができていてやはりそれ以外のものを外してる
#ageが欠損してると死んでる可能性が上がる
data['Predict'][(data.Age.isnull())&(data.Sex == 'female')&(data.Ticket!='X')] = 1
data['Predict'][(data['WCSurvived'].isnull()) & (data.Sex == 'female') & (data.Deck != 'N') ] = 0
data['Predict'][(data['WCSurvived'].isnull()) & (data.Sex == 'female') & (data.Pclass == 1) ] = 0
data['Predict'][(data.Sex == 'female') & (data.Pclass ==1)] = 0
#WC_count>=2だと生きている
data['Predict'][(data.Sex == 'female') & (data.Pclass ==2)&(data.Deck == 'F')] = 0

In [31]:
( data.loc[891:1308, 'Predict'] == data.loc[891:1308, 'Perished'] ).sum() /418

0.8253588516746412

In [32]:
(data.loc[891:1308, 'Predict'][data.Sex == 'male'] == data.loc[891:1308, 'Perished'][data.Sex == 'male']).sum() /266

0.8345864661654135

In [33]:
(data.loc[891:1308, 'Predict'][data.Sex == 'female'] == data.loc[891:1308, 'Perished'][data.Sex == 'female']).sum() / 152

0.8092105263157895

In [34]:
passengers_ID = test_data['PassengerId']

In [35]:
submission = pd.DataFrame({
        "PassengerId": passengers_ID,
        "Perished": data.loc[891:1308, 'Predict'].values.astype(int)
    })

In [36]:
submission

,PassengerId,Perished
0,892,1
1,893,0
2,894,1
3,895,1
4,896,0
5,897,1
6,898,1
7,899,1
8,900,0
9,901,1


In [37]:
submission.to_csv('submission6_14_3:00',index = False)

In [38]:
data.to_csv('data_new.csv',index = False)